In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,explode,split

In [2]:
spark = SparkSession.builder.appName("textSearch").getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/19 15:28:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = [
    ("Spark is amazing and Spark is fast",),
    ("Big data processing is powerful",),
    ("Machine learning with Spark is fun",),
    ("Data science and big data are related",)
]

df = spark.createDataFrame(data,["text"])

In [4]:
df.show()

+--------------------+
|                text|
+--------------------+
|Spark is amazing ...|
|Big data processi...|
|Machine learning ...|
|Data science and ...|
+--------------------+



In [9]:
testWord = df.text.split()

TypeError: 'Column' object is not callable

In [17]:
words_df = df.select(explode(split(col("text")," ")).alias("word"))

In [18]:
words_df.show()

+----------+
|      word|
+----------+
|     Spark|
|        is|
|   amazing|
|       and|
|     Spark|
|        is|
|      fast|
|       Big|
|      data|
|processing|
|        is|
|  powerful|
|   Machine|
|  learning|
|      with|
|     Spark|
|        is|
|       fun|
|      Data|
|   science|
+----------+
only showing top 20 rows



In [21]:
from pyspark.sql.functions import count

In [22]:
words_count_df = words_df.groupBy("word").agg(count("word").alias("countWords"))

In [25]:
words_count_df.orderBy(col("countWords").desc()).show()

+----------+----------+
|      word|countWords|
+----------+----------+
|        is|         4|
|     Spark|         3|
|      data|         2|
|       and|         2|
|   amazing|         1|
|  powerful|         1|
|      fast|         1|
|processing|         1|
|  learning|         1|
|   Machine|         1|
|       Big|         1|
|      Data|         1|
|       big|         1|
|       fun|         1|
|      with|         1|
|       are|         1|
|   related|         1|
|   science|         1|
+----------+----------+



In [26]:
words_count_df.createOrReplaceTempView("words_count")

In [27]:
words_df.createOrReplaceTempView("words")

In [31]:
spark.sql("""
            with countWord as (
            select word,count(word) as words_count from words group by word            
            ),
            rankWordCount as (
            select word,words_count,rank() over(order by words_count desc) as rank_count_words from countWord
            )
            select word,words_count,rank_count_words from rankWordCount where rank_count_words <= 10
""").show()

25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+-----------+----------------+
|      word|words_count|rank_count_words|
+----------+-----------+----------------+
|        is|          4|               1|
|     Spark|          3|               2|
|       and|          2|               3|
|      data|          2|               3|
|   amazing|          1|               5|
|      fast|          1|               5|
|       Big|          1|               5|
|  powerful|          1|               5|
|processing|          1|               5|
|  learning|          1|               5|
|       fun|          1|               5|
|   Machine|          1|               5|
|      with|          1|               5|
|       are|          1|               5|
|   related|          1|               5|
|      Data|          1|               5|
|   science|          1|               5|
|       big|          1|               5|
+----------+-----------+----------------+



25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/19 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/19 1

In [ ]:
spark.sql("""
            with countWord as (
            select word,count(word) as words_count from words group by word            
            ),
            rankWordCount as (
            select word,words_count,rank() over(order by words_count desc) as rank_count_words from countWord
            )
            select word,words_count,rank_count_words from rankWordCount where rank_count_words <= 10
""").show()